In [1]:
import os
import shutil
import random

In [2]:
images_folder = "./part-2"
annotations_folder = "./part-2 labels"
labeled_images_folder = "./labelled_images"
unlabeled_images_folder = "./unlabelled_images"

In [3]:
os.makedirs(labeled_images_folder, exist_ok=True)
os.makedirs(unlabeled_images_folder, exist_ok=True)

In [4]:
all_images = set(os.listdir(images_folder))
all_annotations = {os.path.splitext(f)[0] for f in os.listdir(annotations_folder) if f.endswith('.json')}


In [5]:
for image_file in all_images:
    image_name, _ = os.path.splitext(image_file)
    if image_name in all_annotations:
        # Move labeled images
        shutil.copy(os.path.join(images_folder, image_file), os.path.join(labeled_images_folder, image_file))
    else:
        # Move unlabeled images
        shutil.copy(os.path.join(images_folder, image_file), os.path.join(unlabeled_images_folder, image_file))

print("Separation complete. Labeled and unlabeled images are moved to respective folders.")

Separation complete. Labeled and unlabeled images are moved to respective folders.


In [6]:
# Paths for splitting
train_folder = "./train"
val_folder = "./val"
train_annotations_folder = "./train-labels"
val_annotations_folder = "./val-labels"

# Create folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(train_annotations_folder, exist_ok=True)
os.makedirs(val_annotations_folder, exist_ok=True)


In [7]:
labeled_images = os.listdir(labeled_images_folder)

# Create a list of tuples (image_file, annotation_file)
labeled_image_tuples = [(image_file, f"{os.path.splitext(image_file)[0]}.json") for image_file in labeled_images if os.path.exists(os.path.join(annotations_folder, f"{os.path.splitext(image_file)[0]}.json"))]

# Shuffle the list for randomness
random.shuffle(labeled_image_tuples)

In [8]:
# Define split ratio
split_ratio = 0.8  # 80% for training, 20% for validation
split_index = int(len(labeled_image_tuples) * split_ratio)

# Split into training and validation sets
train_images = labeled_image_tuples[:split_index]
val_images = labeled_image_tuples[split_index:]


In [9]:
for image_file, annotation_file in train_images:
    shutil.copy(os.path.join(labeled_images_folder, image_file), os.path.join(train_folder, image_file))
    shutil.copy(os.path.join(annotations_folder, annotation_file), os.path.join(train_annotations_folder, annotation_file))

for image_file, annotation_file in val_images:
    shutil.copy(os.path.join(labeled_images_folder, image_file), os.path.join(val_folder, image_file))
    shutil.copy(os.path.join(annotations_folder, annotation_file), os.path.join(val_annotations_folder, annotation_file))

print("Splitting complete. Training and validation sets are ready.")

Splitting complete. Training and validation sets are ready.


In [10]:
from labelme2coco import convert

# Paths for training data
train_labelme_folder = "./train-labels"  # Folder containing Labelme JSON files for training
train_output_coco_file = "train-annotations.json"  # Output COCO file for training

# Convert Labelme annotations to COCO format for training set
convert(train_labelme_folder, train_output_coco_file)

# Paths for validation data
val_labelme_folder = "./val-labels"  # Folder containing Labelme JSON files for validation
val_output_coco_file = "val-annotations.json"  # Output COCO file for validation

# Convert Labelme annotations to COCO format for validation set
convert(val_labelme_folder, val_output_coco_file)

print("\nConversion to COCO format completed for both training and validation sets.")

There are 376 listed files in folder ./train-labels.


Converting labelme annotations to COCO format: 100%|█████████████████████████████████| 376/376 [00:04<00:00, 84.79it/s]
11/12/2024 14:53:10 - INFO - labelme2coco -   Converted annotations in COCO format is exported to train-annotations.json\dataset.json


There are 94 listed files in folder ./val-labels.


Converting labelme annotations to COCO format: 100%|███████████████████████████████████| 94/94 [00:01<00:00, 82.38it/s]
11/12/2024 14:53:12 - INFO - labelme2coco -   Converted annotations in COCO format is exported to val-annotations.json\dataset.json



Conversion to COCO format completed for both training and validation sets.


In [12]:
import os

def find_unlabeled_images(images_folder, labels_folder, image_extension=".jpg", label_extension=".json"):
    # Get a set of all image file names (without extension)
    image_files = set(os.path.splitext(f)[0] for f in os.listdir(images_folder) if f.endswith(image_extension))

    # Get a set of all label file names (without extension)
    label_files = set(os.path.splitext(f)[0] for f in os.listdir(labels_folder) if f.endswith(label_extension))

    # Find images without a corresponding label
    unlabeled_images = image_files - label_files

    if unlabeled_images:
        print(f"Images without corresponding labels ({len(unlabeled_images)}):")
        for img in unlabeled_images:
            print(img + image_extension)
    else:
        print("All images have corresponding labels.")

# Paths to your image and label folders
images_folder = "./train"
labels_folder = "./train-labels"

# Call the function to find and print images without corresponding labels
find_unlabeled_images(images_folder, labels_folder)


All images have corresponding labels.


In [13]:
import json
import os

def filter_coco_file(coco_file_path, images_folder):
    # Load the COCO file
    with open(coco_file_path, 'r') as f:
        coco_data = json.load(f)

    # Filter out images that do not exist in the images folder
    filtered_images = []
    filtered_annotations = []

    existing_image_paths = set(os.listdir(images_folder))
    print(f"Existing image files in folder: {existing_image_paths}")

    for image in coco_data['images']:
        image_file_name = os.path.basename(image['file_name'])
        if image_file_name in existing_image_paths:
            filtered_images.append(image)
            # Keep annotations for the existing image
            image_id = image['id']
            filtered_annotations.extend(
                [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]
            )
        else:
            print(f"Image not found: {image_file_name}")

    # Update the COCO data with the filtered lists
    coco_data['images'] = filtered_images
    coco_data['annotations'] = filtered_annotations

    # Save the cleaned COCO file
    cleaned_coco_file_path = coco_file_path.replace('.json', '_cleaned.json')
    with open(cleaned_coco_file_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

    print(f"Filtered {len(filtered_images)} images and {len(filtered_annotations)} annotations.")
    print(f"Filtered COCO file saved at: {cleaned_coco_file_path}")

# Paths to the original COCO files and the folders containing images
train_images_folder = "./train"
val_images_folder = "./val"

# Filter the COCO files for both training and validation sets
filter_coco_file("./dataset-3.json", train_images_folder)
filter_coco_file("./dataset-4.json", val_images_folder)


Existing image files in folder: {'IMG_0954(1).JPG', 'DSC05958.JPG', '20240630_110102AM_Sunday_ByGPSMapCamera.jpg', 'IMG_0066.JPG', 'IMG_0069.JPG', 'IMG_0922.JPG', 'IMG_0223.JPG', '20240630_121642PM_Sunday_ByGPSMapCamera.jpg', 'IMG_0731.JPG', 'IMG_0184.JPG', 'IMG_0439.JPG', 'IMG_0650.JPG', 'DSC06083.JPG', 'IMG_0818(1).JPG', '20240630_102546AM_Sunday_ByGPSMapCamera.jpg', 'IMG_0459.JPG', 'IMG_0922(1).JPG', 'IMG_0931(1).JPG', 'IMG_0809.JPG', 'IMG_0862.JPG', '20240701_123050PM_Monday_ByGPSMapCamera.jpg', '20240701_122849PM_Monday_ByGPSMapCamera.jpg', 'IMG_0967.JPG', 'DSC05996.JPG', 'IMG_0887.JPG', '20240630_105554AM_Sunday_ByGPSMapCamera.jpg', 'IMG_0790(1).JPG', 'DSC06046.JPG', '20240630_105940AM_Sunday_ByGPSMapCamera.jpg', 'IMG_0690.JPG', 'IMG_0948(1).JPG', 'IMG_0975.JPG', 'IMG_1011.JPG', 'IMG_0883.JPG', 'IMG_0840.JPG', 'DSC05938.JPG', 'IMG_0914(1).JPG', '20240630_121534PM_Sunday_ByGPSMapCamera.jpg', '20240630_111917AM_Sunday_ByGPSMapCamera.jpg', 'IMG_0245.JPG', 'IMG_0693.JPG', 'IMG_0967(1

In [14]:
import detectron2
print("Detectron2 is installed successfully!")


Detectron2 is installed successfully!


In [15]:
from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import os
import detectron2.utils.logger as detectron2_logger
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
import cv2

In [22]:
train_images_path = "./train"
train_coco_annotations = "./dataset-3_cleaned.json"
val_images_path = "./val"
val_coco_annotations = "./dataset-4_cleaned.json"

In [23]:
if "leaf_dataset_train" in DatasetCatalog.list(): # Updated line to fix the error
    # If registered, remove the existing registration
    DatasetCatalog.remove("leaf_dataset_train")
    MetadataCatalog.remove("leaf_dataset_train")
if "leaf_dataset_val" in DatasetCatalog.list(): # Updated line to fix the error
    # If registered, remove the existing registration
    DatasetCatalog.remove("leaf_dataset_val")
    MetadataCatalog.remove("leaf_dataset_val")

In [24]:
register_coco_instances("leaf_dataset_train", {}, train_coco_annotations, train_images_path)
register_coco_instances("leaf_dataset_val", {}, val_coco_annotations, val_images_path)

In [25]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("leaf_dataset_train",)
cfg.DATASETS.TEST = ("leaf_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Set to 3 if you have 3 classes


In [26]:
cfg.OUTPUT_DIR = "./detectron-output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [27]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

# Start training
trainer.train()

11/12/2024 15:00:25 - INFO - detectron2.engine.defaults -   Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
     

In [28]:
# Validation Code
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Setup the evaluator and validation data loader
evaluator = COCOEvaluator("leaf_dataset_val", cfg, False, output_dir="./detectron-output")
val_loader = build_detection_test_loader(cfg, "leaf_dataset_val")

# Evaluate the model
evaluation_results = inference_on_dataset(trainer.model, val_loader, evaluator)
print("Validation Results:", evaluation_results)


11/12/2024 15:19:25 - WARNING - detectron2.evaluation.coco_evaluation -   COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
11/12/2024 15:19:25 - WARNING - detectron2.data.datasets.coco -   
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

11/12/2024 15:19:25 - INFO - detectron2.data.datasets.coco -   Loaded 94 images in COCO format from ./dataset-4_cleaned.json
11/12/2024 15:19:25 - INFO - detectron2.data.dataset_mapper -   [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
11/12/2024 15:19:25 - INFO - detectron2.data.common -   Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
11/12/2024 15:19:25 - INFO - detectron2.data.common -   Serializing 94 elements to byte tensors and concatenating them all ...
11/12/2024 15:19:25 - INFO - detectron2.data.common -   Seria

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.637
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.876
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.755
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.598
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.751
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.723
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.729
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.673
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50:0.

In [14]:
# # Prediction Code
# import os
# import cv2
# import torch
# from detectron2.utils.visualizer import Visualizer
# from detectron2.config import get_cfg
# from detectron2.engine import DefaultPredictor
# from detectron2 import model_zoo
# from detectron2.data import MetadataCatalog

# # Define class names (mapping from index to class name)
# class_names = ["turmeric", "maize"]

# # Setup configuration
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = "./detectron-output/model_final.pth"  # Path to the trained weights
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for predictions
# cfg.MODEL.DEVICE = "cuda"  # Use "cpu" if you don't have a GPU
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Number of classes in the dataset

# # Initialize the predictor
# test_images_folder="./unlabelled_images"

# # Iterate through the test images
# for image_name in os.listdir(test_images_folder):
#     if image_name.endswith(('.png', '.jpg', '.jpeg')):  # Check for valid image formats
#         image_path = os.path.join(test_images_folder, image_name)
#         image = cv2.imread(image_path)

#         # Perform inference
#         outputs = predictor(image)

#         # Get the predicted class indices
#         pred_classes = outputs["instances"].pred_classes.cpu().numpy()

#         # Map the predicted indices to class names
#         predicted_labels = [class_names[i] for i in pred_classes]

#         # Visualize the results
#         v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
#         v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

#         # Save the output image with predictions
#         output_image_path = os.path.join(output_folder, image_name)
#         cv2.imwrite(output_image_path, v.get_image()[:, :, ::-1])  # Convert BGR to RGB for saving

#         # Print or log the predicted labels
#         print(f"Predicted labels for {image_name}: {predicted_labels}")


In [29]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Define your class names
classes = ["maize", "turmeric"]

if "leaf_dataset_train" in DatasetCatalog.list(): # Updated line to fix the error
    # If registered, remove the existing registration
    DatasetCatalog.remove("leaf_dataset_train")
    MetadataCatalog.remove("leaf_dataset_train")
if "leaf_dataset_val" in DatasetCatalog.list(): # Updated line to fix the error
    # If registered, remove the existing registration
    DatasetCatalog.remove("leaf_dataset_val")
    MetadataCatalog.remove("leaf_dataset_val")


# Register the datasets
for d in ["train", "val"]:
    DatasetCatalog.register(
        f"leaf_dataset_{d}",
        lambda d=d: load_coco_json(
            f"./dataset-{3 if d == 'train' else 4}_cleaned.json",
            f"./{d}-images-rcnn",
            f"leaf_dataset_{d}"
        )
    )
    MetadataCatalog.get(f"leaf_dataset_{d}").set(thing_classes=classes)

# Update the metadata
leaf_metadata = MetadataCatalog.get("leaf_dataset_train")


In [30]:
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
import cv2
import os
from tqdm import tqdm

# Setup configuration for prediction
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "./detectron-output/model_final.pth"  # Path to your trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for prediction
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Update the number of classes to match your dataset
cfg.MODEL.DEVICE = "cuda"  # or "cpu" if no GPU is available

predictor = DefaultPredictor(cfg)

# Use the correct metadata for visualization
leaf_metadata = MetadataCatalog.get("leaf_dataset_train")

test_images_folder = "./unlabelled_images"
output_folder = "./output-predictions"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Predict for each image and track progress with tqdm
for image_name in tqdm(os.listdir(test_images_folder), desc="Processing Images"):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_images_folder, image_name)
        image = cv2.imread(image_path)

        # Perform inference
        outputs = predictor(image)

        # Visualize results
        v = Visualizer(image[:, :, ::-1], metadata=leaf_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Save the visualized output
        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, v.get_image()[:, :, ::-1])  # Convert BGR to RGB for saving


11/12/2024 15:20:51 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from ./detectron-output/model_final.pth ...
11/12/2024 15:20:51 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from ./detectron-output/model_final.pth ...
C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

In [31]:
#On test set

import os
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo

# Setup configuration for prediction
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "./detectron-output/model_final.pth"  # Path to your trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for prediction
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Update the number of classes to match your dataset
cfg.MODEL.DEVICE = "cuda"  # or "cpu" if no GPU is available

predictor = DefaultPredictor(cfg)

# Folder paths
test_images_folder = "./unlabelled_images"  # Path to your test images folder
output_crops_folder = "./cropped-leaves"  # Path to save cropped shapes

# Create the main output directory if it doesn't exist
if not os.path.exists(output_crops_folder):
    os.makedirs(output_crops_folder)

# Create subfolders for each class
class_names = ["maize", "turmeric"]
for class_name in class_names:
    class_folder = os.path.join(output_crops_folder, class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

# Minimum confidence threshold
confidence_threshold = 0.7  # 70% confidence

# Iterate over each test image
for image_name in os.listdir(test_images_folder):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_images_folder, image_name)
        image = cv2.imread(image_path)

        # Perform inference
        outputs = predictor(image)

        # Get predicted masks, scores, and classes
        masks = outputs["instances"].pred_masks.cpu().numpy()
        scores = outputs["instances"].scores.cpu().numpy()
        classes = outputs["instances"].pred_classes.cpu().numpy()  # Get predicted classes

        # Crop and save each detected leaf using the masks if the score is above the threshold
        for i, (mask, score, cls) in enumerate(zip(masks, scores, classes)):
            if score >= confidence_threshold:
                # Create a binary mask
                binary_mask = mask.astype(np.uint8) * 255

                # Find contours of the mask
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Create a blank image for cropping
                crop_mask = np.zeros_like(image)

                # Draw the contour on the blank image
                cv2.drawContours(crop_mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

                # Mask the original image
                cropped_leaf = cv2.bitwise_and(image, crop_mask)

                # Get bounding box for the contour to crop the relevant part
                x, y, w, h = cv2.boundingRect(contours[0])
                cropped_leaf = cropped_leaf[y:y+h, x:x+w]

                # Determine the class name based on the predicted class
                class_name = class_names[cls]  # Assuming classes are 0 for turmeric and 1 for maize

                # Create a subfolder path for the class
                class_folder = os.path.join(output_crops_folder, class_name)

                # Save the cropped shape
                crop_filename = f"{os.path.splitext(image_name)[0]}_shape_{i}_score_{int(score*100)}.png"
                crop_path = os.path.join(class_folder, crop_filename)
                cv2.imwrite(crop_path, cropped_leaf)

                print(f"Cropped leaf saved at: {crop_path} with score: {score:.2f}")


11/12/2024 15:38:04 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from ./detectron-output/model_final.pth ...
11/12/2024 15:38:04 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from ./detectron-output/model_final.pth ...
C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

Cropped leaf saved at: ./cropped-leaves\maize\20240630_101020AM_Sunday_ByGPSMapCamera_shape_0_score_98.png with score: 0.99
Cropped leaf saved at: ./cropped-leaves\maize\20240630_101020AM_Sunday_ByGPSMapCamera_shape_1_score_98.png with score: 0.98
Cropped leaf saved at: ./cropped-leaves\maize\20240630_101020AM_Sunday_ByGPSMapCamera_shape_2_score_82.png with score: 0.82
Cropped leaf saved at: ./cropped-leaves\maize\20240630_101020AM_Sunday_ByGPSMapCamera_shape_3_score_74.png with score: 0.75
Cropped leaf saved at: ./cropped-leaves\maize\IMG_0697_shape_0_score_98.png with score: 0.98
Cropped leaf saved at: ./cropped-leaves\maize\IMG_0697_shape_1_score_98.png with score: 0.98
Cropped leaf saved at: ./cropped-leaves\maize\IMG_0697_shape_2_score_96.png with score: 0.97
Cropped leaf saved at: ./cropped-leaves\maize\IMG_0697_shape_3_score_95.png with score: 0.96
Cropped leaf saved at: ./cropped-leaves\maize\IMG_0806(1)_shape_0_score_98.png with score: 0.98
Cropped leaf saved at: ./cropped-lea

In [32]:
#On train set

import os
import json

# Define the paths
labels_folder = "./train-labels"
images_folder = './train'

# Loop through each JSON file in the labels folder
for filename in os.listdir(labels_folder):
    if filename.endswith('.json'):
        # Full path to the JSON file
        json_path = os.path.join(labels_folder, filename)

        # Open and load the JSON file
        with open(json_path, 'r') as file:
            data = json.load(file)

        # Update the imagePath to include the full path to the images folder
        image_filename = data.get('imagePath', '')
        new_image_path = os.path.join(images_folder, image_filename)

        # Update the imagePath in the JSON data
        data['imagePath'] = new_image_path

        # Save the updated JSON file
        with open(json_path, 'w') as file:
            json.dump(data, file, indent=4)

        print(f'Updated {filename}')

print("All JSON files have been updated.")


Updated 20240630_100802AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101025AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101239AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101515AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101523AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101925AM_Sunday_ByGPSMapCamera.json
Updated 20240630_101929AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102046AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102226AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102253AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102536AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102546AM_Sunday_ByGPSMapCamera.json
Updated 20240630_102847AM_Sunday_ByGPSMapCamera.json
Updated 20240630_103114AM_Sunday_ByGPSMapCamera.json
Updated 20240630_103118AM_Sunday_ByGPSMapCamera.json
Updated 20240630_105554AM_Sunday_ByGPSMapCamera.json
Updated 20240630_105656AM_Sunday_ByGPSMapCamera.json
Updated 20240630_105732AM_Sunday_ByGPSMapCamera.json
Updated 20240630_105735AM_Sunday_ByGPSMapCamer

In [37]:
#Training set

import os
import cv2
import json
import numpy as np

# Folder paths
labelme_folder = "./train-labels"  # Path to your Labelme JSON files
image_folder = "./train"  # Path where the images are stored
output_crops_folder = "./cropped-leaves-2"  # Path to save cropped shapes

# Create the main output directory if it doesn't exist
if not os.path.exists(output_crops_folder):
    os.makedirs(output_crops_folder)

# Create subfolders for each class
class_names = ["maize", "tumeric"]
for class_name in class_names:
    class_folder = os.path.join(output_crops_folder, class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

# Iterate over each JSON file in the Labelme folder
# Iterate over each JSON file in the Labelme folder
for json_file in os.listdir(labelme_folder):
    if json_file.endswith('.json'):
        json_path = os.path.join(labelme_folder, json_file)

        # Load the JSON file
        with open(json_path) as f:
            data = json.load(f)

        # Modify the imagePath to point to the correct folder
        original_image_path = data['imagePath']
        # Extract the filename only and ignore the original path structure
        modified_image_path = os.path.basename(original_image_path)  
        data['imagePath'] = modified_image_path

        # Load the image using the corrected path in the `./train` folder
        image_path = os.path.join(image_folder, modified_image_path)
        image = cv2.imread(image_path)

        # Check if the image was loaded successfully
        if image is None:
            print(f"Warning: Unable to load image at {image_path}")
            continue  # Skip this JSON file and move to the next one

        # Process each shape in the JSON
        for shape in data['shapes']:
            # Extract polygon points
            points = np.array(shape['points'], dtype=np.int32)
            class_name = shape['label']

            # Create a binary mask from the polygon
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 1)  # Fill the polygon in the mask

            # Find contours of the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Crop and save each detected leaf using the mask
            for i, contour in enumerate(contours):
                # Create a blank image for cropping
                crop_mask = np.zeros_like(image)

                # Draw the contour on the blank image
                cv2.drawContours(crop_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

                # Mask the original image
                cropped_leaf = cv2.bitwise_and(image, crop_mask)

                # Get bounding box for the contour to crop the relevant part
                x, y, w, h = cv2.boundingRect(contour)
                cropped_leaf = cropped_leaf[y:y+h, x:x+w]

                # Determine the class folder based on the label
                if class_name in class_names:
                    class_folder = os.path.join(output_crops_folder, class_name)

                    # Save the cropped shape
                    crop_filename = f"{os.path.splitext(json_file)[0]}_shape_{i}.png"
                    crop_path = os.path.join(class_folder, crop_filename)
                    cv2.imwrite(crop_path, cropped_leaf)

                    print(f"Cropped leaf saved at: {crop_path} for class: {class_name}")

        # Save the modified JSON file with the updated imagePath
        with open(json_path, 'w') as f:
            json.dump(data, f, indent=4)

print("Processing complete.")


Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_100802AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_100802AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_100802AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101025AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101025AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101025AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101239AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101925AM_Sunday_ByGPSMapCamera_shape_0.png for class: maize
Cropped leaf saved at: ./cropped-leaves-2\maize\20240630_101925A